In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder

np.random.seed(42)

# Wczytanie danych
data = pd.read_csv('star_classification.csv', delimiter=",")

# Wybrane cechy
features = ['alpha','delta','u','g','r','i','z','redshift']

# Usuwanie wartości odstających
for col in features:
    mean = data[col].mean()
    std = data[col].std()
    data = data[(data[col] >= mean - 3*std) & (data[col] <= mean + 3*std)]

# Kodowanie klas
le = LabelEncoder()
data['class_encoded'] = le.fit_transform(data['class'])

# Tworzymy X i y (one-hot)
X = data[features].values
y_encoded = data['class_encoded'].values
y = np.eye(3)[y_encoded]  # one-hot

# Łączymy X i y do wspólnej macierzy
combined = np.concatenate((X, y), axis=1)

def split_data(data, train_ratio=0.8):
    np.random.shuffle(data)
    train_size = int(len(data) * train_ratio)
    train_data = data[:train_size]
    test_data = data[train_size:]
    return train_data, test_data

train_data, test_data = split_data(combined)

# Balansowanie tylko zbioru treningowego
train_df = pd.DataFrame(train_data, columns=features + ['c0','c1','c2'])
train_df['class'] = np.argmax(train_df[['c0','c1','c2']].values, axis=1)
min_count = train_df['class'].value_counts().min()
balanced_train_df = pd.concat([
    df.sample(min_count, random_state=42)
    for _, df in train_df.groupby('class')
])
train_data = balanced_train_df.drop(columns='class').values

# Oddzielamy cechy i klasy
X_train = train_data[:, :len(features)]
y_train = train_data[:, len(features):]
X_test = test_data[:, :len(features)]
y_test = test_data[:, len(features):]

# Skalowanie cech
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# etykiety numeryczne do metryk
y_train_lbl = np.argmax(y_train, axis=1)
y_test_lbl = np.argmax(y_test, axis=1)

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

np.random.seed(42)

# ===============================
# FUNKCJE AKTYWACJI
# ===============================
def relu(x, derivative=False):
    if derivative:
        return (x > 0).astype(float)
    return np.maximum(0, x)

def softmax(x):
    exp_x = np.exp(x - np.max(x, axis=1, keepdims=True))
    return exp_x / np.sum(exp_x, axis=1, keepdims=True)

# ===============================
# INICJALIZACJA WAG + BIASÓW
# ===============================
def initialize_parameters(layer_sizes):
    weights, biases = [], []
    for i in range(len(layer_sizes) - 1):
        W = np.random.randn(layer_sizes[i], layer_sizes[i + 1]) * np.sqrt(2 / layer_sizes[i])
        b = np.zeros((1, layer_sizes[i + 1]))
        weights.append(W)
        biases.append(b)
    return weights, biases

# ===============================
# FORWARD / BACKWARD
# ===============================
def forward(X, weights, biases):
    activations = [X]
    for W, b in zip(weights[:-1], biases[:-1]):
        A = relu(activations[-1] @ W + b)
        activations.append(A)
    A_out = softmax(activations[-1] @ weights[-1] + biases[-1])
    activations.append(A_out)
    return activations

def backward(activations, weights, y):
    deltas = [activations[-1] - y]
    for i in reversed(range(len(weights) - 1)):
        delta = deltas[0] @ weights[i + 1].T
        delta *= relu(activations[i + 1], derivative=True)
        deltas.insert(0, delta)

    grads_W, grads_b = [], []
    for i in range(len(weights)):
        grads_W.append(activations[i].T @ deltas[i] / len(y))
        grads_b.append(np.mean(deltas[i], axis=0, keepdims=True))
    return grads_W, grads_b

# ===============================
# TRENING
# ===============================
def train(X, y, hidden_layers, lr, epochs,
          optimizer="sgd", momentum=0.9, beta1=0.9, beta2=0.999):
    t = 0
    layer_sizes = [X.shape[1]] + hidden_layers + [y.shape[1]]
    weights, biases = initialize_parameters(layer_sizes)

    vW = [np.zeros_like(w) for w in weights]
    vb = [np.zeros_like(b) for b in biases]
    mW = [np.zeros_like(w) for w in weights]
    mb = [np.zeros_like(b) for b in biases]
    sW = [np.zeros_like(w) for w in weights]
    sb = [np.zeros_like(b) for b in biases]
    eps = 1e-8

    for _ in range(epochs):
        activations = forward(X, weights, biases)
        grads_W, grads_b = backward(activations, weights, y)

        for i in range(len(weights)):
            if optimizer == "sgd":
                weights[i] -= lr * grads_W[i]
                biases[i] -= lr * grads_b[i]

            elif optimizer == "momentum":
                vW[i] = momentum * vW[i] - lr * grads_W[i]
                vb[i] = momentum * vb[i] - lr * grads_b[i]
                weights[i] += vW[i]
                biases[i] += vb[i]

            elif optimizer == "adam":
                t += 1  # zwiększamy licznik iteracji

                # aktualizacja momentów
                mW[i] = beta1 * mW[i] + (1 - beta1) * grads_W[i]
                mb[i] = beta1 * mb[i] + (1 - beta1) * grads_b[i]
                sW[i] = beta2 * sW[i] + (1 - beta2) * (grads_W[i] ** 2)
                sb[i] = beta2 * sb[i] + (1 - beta2) * (grads_b[i] ** 2)

                # korekcja biasu
                mW_hat = mW[i] / (1 - beta1 ** t)
                mb_hat = mb[i] / (1 - beta1 ** t)
                sW_hat = sW[i] / (1 - beta2 ** t)
                sb_hat = sb[i] / (1 - beta2 ** t)

                # update wag i biasów
                weights[i] -= lr * mW_hat / (np.sqrt(sW_hat) + eps)
                biases[i] -= lr * mb_hat / (np.sqrt(sb_hat) + eps)

    return weights, biases

# ===============================
# PREDYKCJA
# ===============================
def predict(X, weights, biases):
    A = X
    for W, b in zip(weights[:-1], biases[:-1]):
        A = relu(A @ W + b)
    A = softmax(A @ weights[-1] + biases[-1])
    return np.argmax(A, axis=1)

def calculate_classification_metrics(y_true, y_pred):
    acc = accuracy_score(y_true, y_pred)
    prec = precision_score(y_true, y_pred, average="macro", zero_division=0)
    rec = recall_score(y_true, y_pred, average="macro", zero_division=0)
    f1 = f1_score(y_true, y_pred, average="macro", zero_division=0)
    return acc, prec, rec, f1

# ---- HIPERPARAMETRY ----
# learning_rates = [0.001, 0.01, 0.05, 0.1]
# epochs_list = [1500, 1200, 900, 500]
# hidden_layer_configs = [
#         [128, 64, 32, 16], [64, 32, 16, 8], [32, 16, 8, 4],
#         [64, 32, 16], [32, 16, 8], [16, 8, 4],
#         [64, 32], [32, 16], [16, 8], [8, 4],
#         [64], [32], [16], [8]
# ]
# optimizers = ["sgd", "momentum", "adam"]
# momentum_values = [0.6, 0.7, 0.8, 0.9]
# repeat = 4

learning_rates = [0.1]
epochs_list = [500]
hidden_layer_configs = [
        [64], [32]
]
optimizers = ["sgd", "momentum"]
momentum_values = [0.9]
repeat = 2

results = []

for hidden_layers_sizes in hidden_layer_configs:  
    for r in range(1, repeat + 1):
        for lr in learning_rates:
            for epochs in epochs_list:
                for optimizer in optimizers:
                    moms = momentum_values if optimizer == "momentum" else [None]
                    for mom in moms:

                        trained_weights, trained_biases = train(
                            X_train, y_train,
                            hidden_layers=hidden_layers_sizes,
                            lr=lr,
                            epochs=epochs,
                            optimizer=optimizer,
                            momentum=mom or 0.0
                        )

                        # predykcje
                        pred_train = predict(X_train, trained_weights, trained_biases)
                        pred_test = predict(X_test, trained_weights, trained_biases)

                        # metryki
                        acc_train, prec_train, rec_train, f1_train = calculate_classification_metrics(y_train_lbl, pred_train)
                        acc_test, prec_test, rec_test, f1_test = calculate_classification_metrics(y_test_lbl, pred_test)

                        # zapis wyników
                        results.append({
                            'hidden_layers': str(hidden_layers_sizes),
                            'optimizer': optimizer,
                            'momentum': mom,
                            'learning_rate': lr,
                            'epochs': epochs,
                            'repeat': r,
                            'acc_train': acc_train,
                            'precision_train': prec_train,
                            'recall_train': rec_train,
                            'f1_train': f1_train,
                            'acc_test': acc_test,
                            'precision_test': prec_test,
                            'recall_test': rec_test,
                            'f1_test': f1_test
                        })

# ===============================
# Zapis do Excela
# ===============================
df_mlp = pd.DataFrame(results)
df_mlp.to_excel('wyniki_mlp_szczegolowe.xlsx', sheet_name='MLP_szczegolowy', index=False)

# Zagregowane wyniki (średnia po powtórzeniach)
summary_mlp = df_mlp.groupby(['hidden_layers', 'optimizer', 'momentum', 'learning_rate', 'epochs']) \
    .agg({
        'acc_train': ['mean', 'min', 'max'],
        'precision_train': ['mean', 'min', 'max'],
        'recall_train': ['mean', 'min', 'max'],
        'f1_train': ['mean', 'min', 'max'],
        'acc_test': ['mean', 'min', 'max'],
        'precision_test': ['mean', 'min', 'max'],
        'recall_test': ['mean', 'min', 'max'],
        'f1_test': ['mean', 'min', 'max']
    }).reset_index()

# Spłaszczamy MultiIndex w kolumnach
summary_mlp.columns = ['_'.join(col).strip('_') for col in summary_mlp.columns.values]

# Teraz zapisujemy do Excela
summary_mlp.to_excel('wyniki_mlp_zagregowane.xlsx', sheet_name='MLP_zagregowany', index=False)

print("Pliki Excel zostały wygenerowane:")
print("1. wyniki_mlp_szczegolowe.xlsx")
print("2. wyniki_mlp_zagregowane.xlsx")


In [3]:
import numpy as np
import pandas as pd
from itertools import product
from sklearn.preprocessing import LabelEncoder
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


# =======================
# Hiperparametry zgodne z Twoim pierwszym kodem
# =======================
# learning_rates = [0.001, 0.01, 0.05, 0.1]
# epochs_list = [1500, 1200, 900, 500]
# hidden_layer_configs = [
#     [128, 64, 32, 16], [64, 32, 16, 8], [32, 16, 8, 4],
#     [64, 32, 16], [32, 16, 8], [16, 8, 4],
#     [64, 32], [32, 16], [16, 8], [8, 4],
#     [64], [32], [16], [8]
# ]
# optimizers = ['sgd', 'momentum']
# momentum_values = [0.6, 0.7, 0.8, 0.9]
# repeat = 4

learning_rates = [0.1]
epochs_list = [500]
hidden_layer_configs = [
        [64], [32]
]
optimizers = ["sgd", "momentum"]
momentum_values = [0.9]
repeat = 2

results = []

# =======================
# Testowanie wszystkich kombinacji
# =======================
for lr, epochs, hidden, opt in product(learning_rates, epochs_list, hidden_layer_configs, optimizers):
    moms = [0.0] if opt == 'sgd' else momentum_values
    for mom in moms:
        for r in range(1, repeat+1):
            mlp = MLPClassifier(
                hidden_layer_sizes=tuple(hidden),
                activation='relu',
                solver='sgd',
                learning_rate_init=lr,
                momentum=mom,
                max_iter=epochs,
                random_state=r,
            )
            mlp.fit(X_train, y_train_lbl)

            # Predykcje
            y_pred_train = mlp.predict(X_train)
            y_pred_test = mlp.predict(X_test)

            # Metryki
            acc_train = accuracy_score(y_train_lbl, y_pred_train)
            acc_test = accuracy_score(y_test_lbl, y_pred_test)
            precision_train = precision_score(y_train_lbl, y_pred_train, average='macro', zero_division=0)
            recall_train = recall_score(y_train_lbl, y_pred_train, average='macro', zero_division=0)
            f1_train = f1_score(y_train_lbl, y_pred_train, average='macro', zero_division=0)

            precision_test = precision_score(y_test_lbl, y_pred_test, average='macro', zero_division=0)
            recall_test = recall_score(y_test_lbl, y_pred_test, average='macro', zero_division=0)
            f1_test = f1_score(y_test_lbl, y_pred_test, average='macro', zero_division=0)

            # Zapis wyników szczegółowych
            results.append({
                "hidden_layers": str(hidden),
                "optimizer": opt,
                "momentum": mom,
                "learning_rate": lr,
                "epochs": epochs,
                "repeat": r,
                "acc_train": acc_train,
                "precision_train": precision_train,
                "recall_train": recall_train,
                "f1_train": f1_train,
                "acc_test": acc_test,
                "precision_test": precision_test,
                "recall_test": recall_test,
                "f1_test": f1_test
            })

# =======================
# Zapis szczegółowych wyników do Excela
# =======================
df_mlp = pd.DataFrame(results)
df_mlp.to_excel('wyniki_mlp_szczegolowe.xlsx', sheet_name='MLP_szczegolowy', index=False)

# =======================
# Zagregowane wyniki (mean/min/max)
# =======================
summary_mlp = df_mlp.groupby(['hidden_layers', 'optimizer', 'momentum', 'learning_rate', 'epochs']) \
    .agg({
        'acc_train': ['mean', 'min', 'max'],
        'precision_train': ['mean', 'min', 'max'],
        'recall_train': ['mean', 'min', 'max'],
        'f1_train': ['mean', 'min', 'max'],
        'acc_test': ['mean', 'min', 'max'],
        'precision_test': ['mean', 'min', 'max'],
        'recall_test': ['mean', 'min', 'max'],
        'f1_test': ['mean', 'min', 'max']
    }).reset_index()

# Spłaszczamy MultiIndex w kolumnach
summary_mlp.columns = ['_'.join(col).strip('_') for col in summary_mlp.columns.values]

# Zapis do Excela
summary_mlp.to_excel('wyniki_mlp_zagregowane.xlsx', sheet_name='MLP_zagregowany', index=False)

print("Pliki Excel zostały wygenerowane:")
print("1. wyniki_mlp_szczegolowe.xlsx")
print("2. wyniki_mlp_zagregowane.xlsx")


Pliki Excel zostały wygenerowane:
1. wyniki_mlp_szczegolowe.xlsx
2. wyniki_mlp_zagregowane.xlsx
